# **Imports**

In [1]:
# !pip install transformers==4.42.4
# !pip install accelerate==0.32.1
# !pip install torch==1.11.0
# !pip install torchvision==0.12.0
# !pip install mlflow=='2.15.1'
# !pip install --upgrade scikit-learn numpy threadpoolctl
# !pip install pyperclip
# # !pip install databricks-vectorsearch
# !pip install num2words
# # pip install CloudPickle==3.0.0
# !pip install optuna
# !pip install grpcio==1.70.0
# !pip install --force datasets

In [2]:
# !mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

# !wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

# !tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

# !pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

import os

# Clone the repository
!git clone https://github.com/nutonomy/nuscenes-devkit.git

# Check if the directory exists before changing to it
if os.path.isdir('nuscenes-devkit'):
    os.chdir('nuscenes-devkit')
    %pip install -e
    %pip install nuscenes-devkit &> /dev/null
    %pip install nuscenes-devkit
    %pip install nuScenes-lidarseg
else:
    print("Directory 'nuscenes-devkit' does not exist.")

# !wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.
# !wget https://www.nuscenes.org/data/nuScenes-lidarseg-mini-v1.0.tar.bz2

# !tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.
# !tar -xf nuScenes-panoptic-v1.0-mini.tar.gz -C /data/sets/nuscenes   # Uncompress the Panoptic nuScenes mini split.

%pip install datasets==2.14.5 torch torchvision torchaudio accelerate
%pip install nuscenes-devkit
%pip install open3d
%pip install open3d pyvirtualdisplay
# %pip install open3d matplotlib

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from sklearn.decomposition import PCA
import open3d as o3d
import numpy as np
import pandas as pd
# from google.colab.patches import cv2_imshow
from nuscenes.nuscenes import NuScenes
import json
from pathlib import Path
from shutil import copyfile
from typing import List, Dict, Union, Callable, Optional
from copy import deepcopy
import numpy as np
from pyquaternion import Quaternion
from scipy.spatial.transform import Rotation as R
from nuscenes.nuscenes import NuScenes
from nuscenes.scripts.export_kitti import transform_matrix
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from pyquaternion import Quaternion
import pdb
from nuscenes.utils.data_classes import LidarPointCloud
import cv2
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
import open3d as o3d
from pyquaternion import Quaternion
import pdb
import urllib.request
import tarfile
from PIL import Image
from transformers import pipeline
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch
import torch.nn as nn
from torch.nn import BCELoss
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import urllib.request
from tqdm import tqdm
import torch.nn.functional as F

fatal: destination path 'nuscenes-devkit' already exists and is not an empty directory.


Note: you may need to restart the kernel to use updated packages.



Usage:   
  C:\ProgramData\anaconda3\envs\nuscenes_env\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  C:\ProgramData\anaconda3\envs\nuscenes_env\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  C:\ProgramData\anaconda3\envs\nuscenes_env\python.exe -m pip install [options] [-e] <vcs project url> ...
  C:\ProgramData\anaconda3\envs\nuscenes_env\python.exe -m pip install [options] [-e] <local project path> ...
  C:\ProgramData\anaconda3\envs\nuscenes_env\python.exe -m pip install [options] <archive url/path> ...

-e option requires 1 argument


Note: you may need to restart the kernel to use updated packages.


The syntax of the command is incorrect.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement nuScenes-lidarseg (from versions: none)
ERROR: No matching distribution found for nuScenes-lidarseg


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


C:\ProgramData\anaconda3\envs\nuscenes_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from nuscenes.nuscenes import NuScenes
files_dir = "D:/Nuscenes/unified"

# **Functions**

In [4]:
#configuration
num_objects = 3
x_size = 120
y_size = 90
grid_size = 1

In [5]:
device = 0 if torch.cuda.is_available() else -1
detector = pipeline("object-detection", model="facebook/detr-resnet-50", device = device)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using a slow image processor as `use_fast` is unset and a slow processor was saved w

In [6]:
def calc_camera_point_using_pnhole(ann, sensor_channel):
  # Retrieve the camera calibration information
  sensor_data = nusc.get('sample_data',my_sample['data'][sensor_channel])
  cam_intrinsic = nusc.get('calibrated_sensor', sensor_data['calibrated_sensor_token'])
  #calculate distance function
  cam_intrinsic = np.array(nusc.get('calibrated_sensor', sensor_data['calibrated_sensor_token'])['camera_intrinsic'])
  fx = cam_intrinsic[0][0]
  fy = cam_intrinsic[1][1]
  cx = cam_intrinsic[0][2]
  cy = cam_intrinsic[1][2]
  # Extract truck width statistics
  car_stats = stats["vehicle.car"]
  truck_stats = stats["vehicle.truck"]
  person_stats = stats["human.pedestrian.adult"]
  avg_width_meters = {""}
  avg_car_width_meters = float(car_stats["width"].split("±")[0])
  avg_truck_width_meters = float(truck_stats["width"].split("+-")[0])
  avg_person_width_meters = float(person_stats["width"].split("±")[0])
  x_pixel = ann['xmax'] - ann['xmin']
  y_pixel = ann['ymax'] - ann['ymin']
  center_y_pixel = (ann['ymax'] + ann['ymin']) / 2
  center_x_pixel = (ann['xmax'] + ann['xmin']) / 2
  if ann['object']=="person":
    z_camera = (fx * width_stats['human.pedestrian.adult']) / x_pixel
  else:
    z_camera = (fx * width_stats['vehicle.'+ann['object']]) / x_pixel
  x_camera = (z_camera * (center_x_pixel - cx)) / fx
  y_camera = (z_camera * (center_y_pixel - cy)) / fy

  #transform the points into the lidar axis
  transformation_matrix = np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]])
  object_location_camera_transposed = np.array([x_camera, y_camera, z_camera]).reshape(3, 1)
  result = np.dot(transformation_matrix, object_location_camera_transposed)
  camera_to_lidar = result.T.reshape(num_objects)

  # Return a Series instead of individual values
  return pd.Series({'camera_point_x': camera_to_lidar[0], 'camera_point_y': camera_to_lidar[1], 'camera_point_z': camera_to_lidar[2]})


def generate_potential_annotations(sensor_name):
  global detector
  #Load the sensor data from NuScenes
  sensor_data = nusc.get('sample_data', my_sample['data'][sensor_name])
  # Get the image file path
  image_filepath = sensor_data['filename']
  image_fullpath = os.path.join(nusc.dataroot, image_filepath)
  # Check if the image file exists
  if not os.path.exists(image_fullpath):
      print(fr"The file {image_fullpath} does not exist at the provided path.")
      not_found_files.append(image_fullpath)
      return None
  else:
      # Read the image
      image = cv2.imread(image_fullpath)
    
      # Display the image
      # cv2_imshow(image)
  ################################
  # Load the model for detecting potential annotations

  
  # Convert the NumPy array to a PIL Image object
  image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) #added color conversion

  # Perform object detection
  results = detector(image_pil)

  # Create a copy of the image for drawing bounding boxes
  image_with_boxes = image.copy()

  # Iterate through the results and draw bounding boxes
  for result in results:
      xmin_normalized = result['box']['xmin']
      ymin_normalized = result['box']['ymin']
      xmax_normalized = result['box']['xmax']
      ymax_normalized = result['box']['ymax']

      # Denormalize the bounding box coordinates
      image_height, image_width, _ = image.shape #get image dimensions
      xmin = int(xmin_normalized * image_width)
      ymin = int(ymin_normalized * image_height)
      xmax = int(xmax_normalized * image_width)
      ymax = int(ymax_normalized * image_height)

      # Draw the bounding box and label on the image
#       cv2.rectangle(image_with_boxes, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
#       cv2.putText(image_with_boxes, f"{result['label']}: {result['score']:.2f}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
  #########################
    # Load the image object detection pipeline - note the hyphen instead of underscore


  # Convert the NumPy array to a PIL Image object
  image_pil = Image.fromarray(image)

  # Perform object detection on the PIL Image object
  results = detector(image_pil)  # Pass the PIL Image

  potential_annotations = pd.DataFrame()

  # Optionally, visualize the results (requires OpenCV)
  image_with_boxes = image.copy()  # Continue using the NumPy array for visualization
  for result in results:
      xmin_normalized = result['box']['xmin']
      ymin_normalized = result['box']['ymin']
      xmax_normalized = result['box']['xmax']
      ymax_normalized = result['box']['ymax']
      # # Denormalize the bounding box coordinates
      # xmin_pixel = int(xmin_normalized * 1600)
      # ymin_pixel = int(ymin_normalized * 900)
      # xmax_pixel = int(xmax_normalized * 1600)
      # ymax_pixel = int(ymax_normalized * 900)

      result_df = pd.DataFrame([{"object": result['label'],
                                "confidence": result['score'],
                                "xmin": xmin_normalized,
                                "ymin": ymin_normalized,
                                "xmax": xmax_normalized,
                                "ymax": ymax_normalized}])
      potential_annotations = pd.concat([potential_annotations, result_df])
#       cv2.rectangle(image_with_boxes, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
#       cv2.putText(image_with_boxes, f"{result['label']}: {result['score']:.2f}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
  #########################
  ###preprocess potential annotations
  # Display the image with bounding boxes
  #process the data so it will be suitable to the format, select only people and vechiles
  potential_annotations.reset_index(inplace = True, drop = True)
  if potential_annotations.shape[0] ==0:
    return pd.DataFrame()
  potential_annotations['height'] = potential_annotations['ymax'] - potential_annotations['ymin']

  potential_annotations['width'] = potential_annotations['xmax'] - potential_annotations['xmin']
  potential_annotations['num_pixels'] = potential_annotations['height'] * potential_annotations['width']
  potential_annotations = potential_annotations[potential_annotations['object'].isin(['car', 'truck', 'person'])]
  potential_annotations.head()
  potential_annotations.sort_values(by = 'confidence', ascending = False).head()
  sample_token = nusc.get('sample_data', sensor_data['token'])['sample_token']
  ground_truth_annotation_keys= nusc.get('sample', sample_token)['anns']

  potential_annotations = potential_annotations[potential_annotations['object'].isin(["car", "truck", "person"])]
  # Apply the function and assign the resulting columns
  try:
      potential_annotations[['camera_point_x', 'camera_point_y', 'camera_point_z']] = potential_annotations.apply(lambda x: calc_camera_point_using_pnhole(x, sensor_data['channel']), axis=1)
  except ValueError:
    if potential_annotations.shape[0] < 1 :
        print (fr"No detections for image {image_fullpath}")
  potential_annotations['sensor_channel'] = sensor_data['channel']
  return potential_annotations

def cartesian_to_polar(x, y, z):
  """Converts Cartesian coordinates to polar coordinates (translation and rotation).

  Args:
    x: x-coordinate.
    y: y-coordinate.
    z: z-coordinate.

  Returns:
    A dictionary containing the translation (distance) and rotation (azimuth and elevation).
  """

  distance = np.sqrt(x**2 + y**2 + z**2)
  azimuth = np.arctan2(y, x)
  elevation = np.arctan2(z, np.sqrt(x**2 + y**2))

  return {'distance': distance, 'azimuth': azimuth, 'elevation': elevation}

def points_in_box(box: 'Box', points: np.ndarray, wlh_factor: float = 1.0):
    """
    Checks whether points are inside the box.
    Picks one corner as reference (p1) and computes the vector to a target point (v).
    Then for each of the 3 axes, project v onto the axis and compare the length.
    Inspired by: https://math.stackexchange.com/a/1552579
    :param box: <Box>.
    :param points: <np.float: 3, n>.
    :param wlh_factor: Inflates or deflates the box.
    :return: <np.bool: n, >.
    """
    corners = box.corners(wlh_factor=wlh_factor)

    p1 = corners[:, 0]
    p_x = corners[:, 4]
    p_y = corners[:, 1]
    p_z = corners[:, 3]

    i = p_x - p1
    j = p_y - p1
    k = p_z - p1

    v = points - p1.reshape((-1, 1))

    iv = np.dot(i, v)
    jv = np.dot(j, v)
    kv = np.dot(k, v)

    mask_x = np.logical_and(0 <= iv, iv <= np.dot(i, i))
    mask_y = np.logical_and(0 <= jv, jv <= np.dot(j, j))
    mask_z = np.logical_and(0 <= kv, kv <= np.dot(k, k))
    mask = np.logical_and(np.logical_and(mask_x, mask_y), mask_z)

    return mask

###convert camera point to lidar point
def convert_cam_to_lidar(sample, camera_direction, point_to_convert):
  camera_rec = nusc.get('sample_data', sample['data'][camera_direction])
  camera_cs = nusc.get('calibrated_sensor', camera_rec['calibrated_sensor_token'])
  lidar_rec = nusc.get('sample_data', sample['data'][camera_direction])
  lidar_cs = nusc.get('calibrated_sensor', lidar_rec['calibrated_sensor_token'])

  ego_pose = nusc.get('ego_pose', camera_rec['ego_pose_token'])
  ##create camerara extrinsic matrix
  translation_vector = np.array(camera_cs['translation'])
  rotation_matrix =R.from_quat(camera_cs['rotation']).as_matrix()
  extrinsic_matrix = np.eye(4)
  extrinsic_matrix[:3, :3] = rotation_matrix
  ###transform to the vehicle crdinates
  point_vehicle = extrinsic_matrix @ point_to_convert
  ###transform to the lidar coordinates
  T_lidar = np.eye(4)
  T_lidar[:3, :3] = R.from_quat(lidar_cs['rotation']).as_matrix()
  T_lidar[:3, 3] = lidar_cs['translation']
  T_lidar_inv = np.linalg.inv(T_lidar)
  point_lidar = T_lidar_inv @ point_vehicle
  return point_lidar[:3]
# a = convert_cam_to_lidar(my_sample, "CAM_FRONT", np.array([potential_annotations.loc[17]['camera_point_x'], potential_annotations.loc[17]['camera_point_y'], potential_annotations.loc[17]['camera_point_z'], 1]))


def create_grid_map_lidar(ground_truth, grid_size=grid_size):
    """
    Creates a grid map from ground truth point cloud data.

    Args:
        ground_truth: Pandas DataFrame containing 'x', 'y', 'z', and 'intensity' columns.
        grid_size: Size of each grid cell in meters (default: 3).

    Returns:
        A NumPy array representing the grid map, where each cell contains the average intensity.
    """
    if ground_truth.empty or ground_truth['x'].isna().all() or ground_truth['y'].isna().all():
        print("Warning: Ground truth data contains no valid coordinates")
        return None
    # Find min/max coordinates
    min_x = ground_truth['x'].min()
    max_x = ground_truth['x'].max()
    min_y = ground_truth['y'].min()
    max_y = ground_truth['y'].max()
    
#     print ("max_x: ",max_x)
#     print ("min_x: ",min_x)
#     print ("grid_size: ",grid_size)
#     # Calculate grid dimensions
#     display("grid_size: ", grid_size)
#     display("max_x: ", max_x)
#     display("min_x: ", min_x)
#     display("max_x type:", type(max_x))
#     display("min_x type:", type(min_x))
#     display("max_x - min_x:", max_x - min_x)
    x_dim = int(np.ceil((max_x - min_x) / grid_size))
    y_dim = int(np.ceil((max_y - min_y) / grid_size))

    # Initialize the grid map
    grid_map = np.zeros((y_dim, x_dim))

    # Iterate over the points and populate the grid map
    for index, row in ground_truth.iterrows():
        x = row['x']
        y = row['y']
        intensity = row['intensity']

        # Calculate grid cell indices
        grid_x = int(np.floor((x - min_x) / grid_size))
        grid_y = int(np.floor((y - min_y) / grid_size))

        # Check bounds
        if 0 <= grid_x < x_dim and 0 <= grid_y < y_dim:
            # Accumulate intensity values in the grid cell
            grid_map[grid_y, grid_x] += intensity

    # Calculate average intensity for each cell
    point_counts = np.zeros_like(grid_map)

    for index, row in ground_truth.iterrows():
        x = row['x']
        y = row['y']

        # Calculate grid cell indices
        grid_x = int(np.floor((x - min_x) / grid_size))
        grid_y = int(np.floor((y - min_y) / grid_size))

        # Check bounds
        if 0 <= grid_x < x_dim and 0 <= grid_y < y_dim:
            point_counts[grid_y, grid_x] += 1
    grid_map = np.divide(grid_map, point_counts, out=np.zeros_like(grid_map), where=point_counts!=0) # handle divide by zero
    return grid_map

def create_grid_map(potential_annotations, grid_size=3, object_type=None):
    """
    Creates a grid map from object detection data.

    Args:
        potential_annotations: Pandas DataFrame with 'camera_point_x', 'camera_point_y', and 'object' columns.
        grid_size: Size of each grid cell in meters.

    Returns:
        A NumPy array representing the grid map.
    """

    # Find min/max coordinates
#     display("potential_annotations: ", potential_annotations)
    min_x = potential_annotations['camera_point_x'].min()
    max_x = potential_annotations['camera_point_x'].max()
    min_y = potential_annotations['camera_point_y'].min()
    max_y = potential_annotations['camera_point_y'].max()
    # Calculate grid dimensions
    x_dim = int(np.ceil((max_x - min_x) / grid_size))
    y_dim = int(np.ceil((max_y - min_y) / grid_size))

    # Initialize the grid map
    grid_map = np.empty((y_dim, x_dim), dtype=object)  # Use object dtype to store strings
    grid_map[:] = 0  # Initialize with empty strings

    
    potential_annotations = potential_annotations[potential_annotations['object'] == object_type.split('_')[-1]]
    if len(potential_annotations) < 1:
        return torch.zeros(x_size, y_size)
    # Iterate through the detections
    for _, row in potential_annotations.iterrows():
        if len(potential_annotations) ==0:
            grid_map = np.zeros(x_dim, y_dim)
            return grid_map
        x = row['camera_point_x']
        y = row['camera_point_y']
        obj_cols = row[-3:]
        obj = obj_cols[obj_cols == 1].index[0].split("_")[1]
        confidence = row['confidence'] # Extract the confidence value as a float

        # Calculate grid cell indices
        grid_x = int(np.floor((x - min_x) / grid_size))
        grid_y = int(np.floor((y - min_y) / grid_size))

        # Check bounds
        if 0 <= grid_x < x_dim and 0 <= grid_y < y_dim:
            # If the cell is empty or the current object has higher confidence
            if grid_map[grid_y, grid_x] == 0 or confidence > grid_map[grid_y, grid_x]: # Compare confidence values directly
                grid_map[grid_y, grid_x] = confidence
        print (fr"map for {object_type.split('_')[-1]}")
#     plot_optical_grid_map(grid_map)
    return grid_map

def create_ground_truth_map(ground_truth, grid_size=grid_size):
    """
    Creates a ground truth grid map from object detection data.

    Args:
        ground_truth: Pandas DataFrame with 'x', 'y', and 'label' columns.
        grid_size: Size of each grid cell in meters.

    Returns:
        A NumPy array representing the ground truth grid map.
    """
    if ground_truth.empty or ground_truth['x'].isna().all() or ground_truth['y'].isna().all():
        print("Warning: Ground truth data contains no valid coordinates")
        return None, None
    # Find min/max coordinates
    
    min_x = ground_truth['x'].min()
    max_x = ground_truth['x'].max()
    min_y = ground_truth['y'].min()
    max_y = ground_truth['y'].max()
#     print ("max_x: ",max_x)
#     print ("min_x: ",min_x)
#     print ("grid_size: ",grid_size)
    # Calculate grid dimensions
#     display("grid_size: ", grid_size)
#     display("max_x: ", max_x)
#     display("min_x: ", min_x)
#     display("max_x type:", type(max_x))
#     display("min_x type:", type(min_x))
#     display("max_x - min_x:", max_x - min_x)
    # Calculate grid dimensions
    x_dim = int(np.ceil((max_x - min_x) / grid_size))
    y_dim = int(np.ceil((max_y - min_y) / grid_size))

    # Get unique object labels
    unique_labels = {"truck": 0, "car":1, "person":2}
    
    num_objects = len(unique_labels)

    # Initialize the ground truth grid map (one map per object)
    ground_truth_map = np.zeros((num_objects, y_dim, x_dim), dtype=int)

    # Iterate through the detections
    for _, row in ground_truth.iterrows():
        x = row['x']
        y = row['y']
        label = row['label']

        # Calculate grid cell indices
        grid_x = int(np.floor((x - min_x) / grid_size))
        grid_y = int(np.floor((y - min_y) / grid_size))

        # Check bounds
        if 0 <= grid_x < x_dim and 0 <= grid_y < y_dim:
            # Find the object index
            object_index = unique_labels[label]
            # Mark the cell as occupied for the corresponding object
            ground_truth_map[object_index, grid_y, grid_x] = 1
            
    return ground_truth_map, unique_labels

def pad_tensor_to_target(tensor, target_shape):
    """
    Pads a 3D tensor to a target shape using torch.nn.functional.pad, focusing on the last two dimensions.

    Args:
        tensor (torch.Tensor): The 3D tensor to pad.
        target_shape (tuple): The desired shape of the padded tensor (target_depth, target_height, target_width).

    Returns:
        torch.Tensor: The padded tensor.
    """
    if len(tensor.shape) == 2:
        tensor = tensor.unsqueeze(0)  # Add batch dimension
    tensor_depth, tensor_height, tensor_width = tensor.shape
    target_depth, target_height, target_width = target_shape

    pad_height_before = (target_height - tensor_height) // 2
    pad_height_after = target_height - tensor_height - pad_height_before

    pad_width_before = (target_width - tensor_width) // 2
    pad_width_after = target_width - tensor_width - pad_width_before

    pad_depth_before = (target_depth - tensor_depth) // 2
    pad_depth_after = target_depth - tensor_depth - pad_depth_before

    padding = (pad_width_before, pad_width_after, pad_height_before, pad_height_after, pad_depth_before, pad_depth_after)

    # Pad the last three dimensions (height, width, depth)
    padded_tensor = F.pad(tensor, (pad_width_before, pad_width_after, pad_height_before, pad_height_after, pad_depth_before, pad_depth_after))

    return padded_tensor

### Application of the functions

In [7]:
# # Define the file path
# file_path = r"D:\Nuscenes\required scenes.txt"

# # Initialize an empty list to store the rows
# scenes_list = []

# # Open and read the file
# with open(file_path, 'r') as file:
#     # Read each line and add it to the list
#     for line in file:
#         # Strip whitespace and newline characters
#         scenes_list.append(line.strip())
scenes_list = ['c610d0ac2e2045a5972bda66ac42ede3', 'ea0979999bf84ee6a6230b1bde7ec723', "b2119bd765a94b0cb705424800307bbf", 
              '270fe8382f884386b3445143fa946200', '32bae098e2434a09af1e7983e31fbf5c', '2bfb95d8ba3a4c11869f4d6635784640', '29dee0b8210b410a9b3421f0cb39019f', 
               '69a38ccf0894467fbae658d28d4fd10a', '34a9823fac0d4b9db30898e00b5f9f9c','faa975eaad804c85bd8f2da9b30c8d9a','d4d2faa310a748dda508d15c6d5c4244','7bbc6931b39b4ba3834240618fc4a4c2','fee7ac5f5c1e4943877cd20d6e069427','797700339b014fb6aa61c1ea3778d587','d9161e521b1644cea4cb9e3f21ef9f79','19640fe4b6894f97b16e0faad51432b4','2a334f5c4615465f86a250d4758a2d7d','f634de95cc7043b8b38ceaac67d472cf','8d8728879b89405eba3f8ce1af138cb2','4f679c8f8f6d4f5d8466253dda4733ba','cd9db61edff14e8784678abb347cd674','2175d0e84f224ea69907e5c338bde395','9544516643a14926bf750fc71c1b307d','e631037169574c67944176bf079ee75c','bc5daf905b854dbc8944811c4b7c319d','390f81ac97474695a727d688240e6fba','4ed628299b1e45a3a73704e8cf8287a9','add2303ca41d445c8339e05769745fc0','2376fcdd28a44b83962289664cee55b7','16c90eedfc7943a5a4c64e84d18876a2','102b894332464704900264216480a8f7','bfe368f8c6a44efcb323a82f6974a9a6','034dee1695304630b0692da8c1f153fc','7052d21b95fc4bae8761b8d9524f3e42','e8099a6136804f3bb9b38ff94d98eb64','e6f1a7e6218a4737bfedc6af90926b3e','6af9b75e439e4811ad3b04dc2220657a','f97bf749746c4c3a8ad9f1c11eab6444','bd338b912ce9434995b29b6dac9fbf1d' ]
# Print the list to verify
print(scenes_list)

['c610d0ac2e2045a5972bda66ac42ede3', 'ea0979999bf84ee6a6230b1bde7ec723', 'b2119bd765a94b0cb705424800307bbf', '270fe8382f884386b3445143fa946200', '32bae098e2434a09af1e7983e31fbf5c', '2bfb95d8ba3a4c11869f4d6635784640', '29dee0b8210b410a9b3421f0cb39019f', '69a38ccf0894467fbae658d28d4fd10a', '34a9823fac0d4b9db30898e00b5f9f9c', 'faa975eaad804c85bd8f2da9b30c8d9a', 'd4d2faa310a748dda508d15c6d5c4244', '7bbc6931b39b4ba3834240618fc4a4c2', 'fee7ac5f5c1e4943877cd20d6e069427', '797700339b014fb6aa61c1ea3778d587', 'd9161e521b1644cea4cb9e3f21ef9f79', '19640fe4b6894f97b16e0faad51432b4', '2a334f5c4615465f86a250d4758a2d7d', 'f634de95cc7043b8b38ceaac67d472cf', '8d8728879b89405eba3f8ce1af138cb2', '4f679c8f8f6d4f5d8466253dda4733ba', 'cd9db61edff14e8784678abb347cd674', '2175d0e84f224ea69907e5c338bde395', '9544516643a14926bf750fc71c1b307d', 'e631037169574c67944176bf079ee75c', 'bc5daf905b854dbc8944811c4b7c319d', '390f81ac97474695a727d688240e6fba', '4ed628299b1e45a3a73704e8cf8287a9', 'add2303ca41d445c8339e05769

In [8]:
nusc = NuScenes(version='v1.0-trainval', dataroot=files_dir, verbose=True)
scenes_df = pd.DataFrame(scenes_list)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 77.542 seconds.
Reverse indexing ...
Done reverse indexing in 16.2 seconds.


In [9]:
from nuscenes.nuscenes import NuScenes
import logging
import torch
import pandas as pd
import numpy as np
import warnings
import os
import open3d as o3d
from tqdm import tqdm  # Import tqdm for progress bar
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import points_in_box
import transformers

logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message=".*copying from a non-meta parameter.*", category=UserWarning)
# sensor_data = nusc.get('sample_data',my_sample['data'][sensor_channel])

# --- הגדרות CHECKPOINT ---
# 1. נגדיר כל כמה דגימות (samples) נרצה לשמור את ההתקדמות
SAVE_EVERY_N_SAMPLES = 20  # לדוגמה, שמירה כל 20 דגימות
# 2. נגדיר את שם קובץ השמירה
CHECKPOINT_PATH = 'nuscenes_checkpoint1.pth'

# Suppress warnings and logs
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message=".*copying from a non-meta parameter.*", category=UserWarning)

# --- לוגיקת טעינת CHECKPOINT ---
# נבדוק אם קובץ השמירה קיים לפני שמתחילים את הלולאה
if os.path.exists(CHECKPOINT_PATH):
    print(f"✅ נמצא קובץ שמירה. טוען התקדמות מ: {CHECKPOINT_PATH}")
    checkpoint = torch.load(CHECKPOINT_PATH)
    
    # טעינת כל המשתנים השמורים
    scene_idx = checkpoint['scene_idx']
    next_scene = checkpoint['next_scene']
    training_batch = checkpoint['training_batch']
    label_batch = checkpoint['label_batch']
    logs = checkpoint['logs']
    not_found_files = checkpoint['not_found_files']
    scenes_with_missing_files = checkpoint['scenes_with_missing_files']
    
    # נקבע את נקודת ההתחלה של מד ההתקדמות
    pbar_initial = scene_idx
    print(f"הריצה תמשיך מסצנה מספר {scene_idx}.")

else:
    print("❌ לא נמצא קובץ שמירה. מתחיל ריצה חדשה מההתחלה.")
    # אתחול המשתנים כפי שהיו בקוד המקורי
    training_batch = torch.empty(0, num_objects + 1, x_size, y_size)
    label_batch = torch.empty(0, num_objects, x_size, y_size)
    logs = {}
    not_found_files = []
    scenes_with_missing_files = []
    scene_idx = 0
    next_scene = None
    pbar_initial = 0

# Add progress bar for the main loop
total_scenes = scenes_df.shape[0]
with tqdm(total=total_scenes, desc="Processing scenes") as pbar:
    while scene_idx < scenes_df.shape[0]:
        token_idx = scenes_df.iloc[scene_idx,0]
        logging.getLogger("torch.device").setLevel(logging.ERROR)
        my_scene = nusc.get("scene",token_idx )
        if next_scene is None:
            display("my scene: ", my_scene)
            sample_token = my_scene['first_sample_token']
        elif next_scene == '':
            scene_idx += 1
            next_scene = None
            pbar.update(1)  # Update progress bar when moving to next scene
            continue
        else:
            sample_token = next_scene
        print ("current scene is: ", my_scene)
        print ("description: ", my_scene['description'])
        print ("current sample_token: ", sample_token)
        #if we are in the last sample_token of a scene then there will be a keyerror
        my_sample = nusc.get('sample', sample_token)
        ###################################################
        #create a dictionary of objects stats
        stats = {
            "vehicle.truck":{"width":"2.35+-0.34",'len': "6.50±1.56", "height": "2.62±0.68", "lw_aspect": "2.75±0.37"},
                "vehicle.car":{"width":"1.92±0.16",'len': "4.62±0.36", "height": "1.69±0.21", "lw_aspect": "2.41±0.18"},
                "human.pedestrian.adult":{"width":"0.68±0.11", "len": "0.73±0.17", "height": "1.76±0.12", "lw_aspect": "1.08±0.23"}}
        width_stats = {}
        for key, stat in zip(stats.keys(), stats.values()):
            try:
                avg_width_meters = float(stat["width"].split("±")[0])
            except Exception:
                try:
                    avg_width_meters = float(stat["width"].split("+-")[0])
                except Exception:
                    avg_width_meters = None
            width_stats[key] = avg_width_meters
        ### Applications of the functions:
        #generate optical sensor data - applications
        potential_annotations = pd.DataFrame()
        opt_sensors = ['CAM_FRONT', "CAM_BACK_RIGHT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_FRONT_LEFT" ]
        for sensor in opt_sensors:
            sensor_data = generate_potential_annotations(sensor)
            if sensor_data is None:
                print ("scene doesn't have all the files, skipping to the next one")
                scenes_with_missing_files.append(sample_token)
                next_scene = nusc.get('sample',sample_token)['next']
                break
            if sensor_data.shape[0] == 0:
                print ("No detections, skipping to next sensor")
                continue
                
            potential_annotations = pd.concat([potential_annotations, sensor_data])
        if sensor_data is None:
            next_scene = nusc.get('sample',sample_token)['next']
            continue
        if potential_annotations.shape[0] > 0:
            potential_annotations[['lidar_x', 'lidar_y', 'lidar_z']] = potential_annotations.apply(
                lambda row: pd.Series(convert_cam_to_lidar(my_sample, row['sensor_channel'], np.array([row['camera_point_x'], row['camera_point_y'], row['camera_point_z'], 1]))),
                axis=1
            )
            # Example usage with your DataFrame:
            potential_annotations['polar_coordinates'] = potential_annotations.apply(
                lambda row: cartesian_to_polar(row['camera_point_x'], row['camera_point_y'], row['camera_point_z']),
                axis=1
            )
        else:
            next_scene = nusc.get('sample',sample_token)['next']
            continue

        #read lidar data and process it
        sensor = 'LIDAR_TOP'
        lidar_data = nusc.get('sample_data', my_sample['data'][sensor])
        # Get the image file path
        lidar_filepath = lidar_data['filename']
        lidar_filepath = os.path.join(nusc.dataroot, lidar_filepath)
        # nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=False)
        pcd_bin_file = os.path.join(nusc.dataroot, nusc.get('sample_data', my_sample['data'][sensor])['filename'])
        
        pc = LidarPointCloud.from_file(pcd_bin_file)
        bin_pcd = pc.points.T
        bin_pcd_only_axis = bin_pcd.reshape((-1, 4))[:, 0:3]

        o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(bin_pcd_only_axis))
        bin_pcd_with_intensity = bin_pcd.reshape((-1, 4))
        intensity_values = bin_pcd_with_intensity[:,3]
        
        full_lidar_df = pd.DataFrame({
        'x': bin_pcd_only_axis[:, 0],
        'y': bin_pcd_only_axis[:, 1],
        'z': bin_pcd_only_axis[:, 2],
        'intensity': intensity_values
            })
        
        current_dir = os.getcwd()

        dataroot = os.path.join(current_dir,"data","sets","nuscenes")

        tagged_points = pd.DataFrame()

         #first_sample_token = my_scene['first_sample_token']

        sample = nusc.get('sample', sample_token)
        
        
        
        ann_tokens = sample['anns']
        if type(ann_tokens) != list:
            ann_tokens = [ann_tokens]


        for ann_token in ann_tokens:
            ann = nusc.get('sample_annotation',ann_token)
            lidar_data = sample['data']['LIDAR_TOP']

            data_path,boxes,camera_intrinsic = nusc.get_sample_data(lidar_data,selected_anntokens=[ann_token], )
            
            lidar_rec = nusc.get('sample_data',lidar_data)
            
            mask = points_in_box(boxes[0],pc.points[:3])
            object_type =pd.DataFrame({ "key":ann_token, 'text': [ann['category_name']] *  np.sum(mask)})
            ann_points = pc.points[:,mask].T
            ann_points = pd.DataFrame(pc.points[:,mask].T)
            ann_points = pd.concat([object_type, ann_points ], axis = 1).rename(columns={"key":"key", 0: 'x', 1: 'y', 2: 'z', 3: 'intensity', 'text': 'label'})
            tagged_points = pd.concat([ann_points, tagged_points])
        ground_truth = tagged_points

        ###create the ground truth data
        ground_truth = ground_truth[ground_truth['label'].isin(['vehicle.car', 'vehicle.truck', 'human.pedestrian.adult'])]
        lidar_grid = create_grid_map_lidar(full_lidar_df)
        
        if lidar_grid is None:
            print ("scene doesn't have a valid lidar input, skipping to the next scene")
            scenes_with_missing_files.append(sample_token)
            next_scene = nusc.get('sample',sample_token)['next']
            continue

        #apply one hot encoding
        encoded_columns= pd.get_dummies(potential_annotations, columns=['object'], prefix='object', drop_first=False).iloc[:,-3:]
        potential_annotations = pd.concat([potential_annotations, encoded_columns], axis=1)

        #create a grid map for the
        objects_dict = {}
        objects_grid_maps = []
        possible_objects = [col for col in potential_annotations.columns if col.startswith('object_')]
        for i in range(len(possible_objects)):
            objects_dict[possible_objects[i].split("_")[-1]] = i
            display("objects dict: ", objects_dict)
            display("potential_anotations: ",potential_annotations)
            grid_map = create_grid_map(potential_annotations, object_type = possible_objects[i])
            objects_grid_maps.append(grid_map)
        objects_grid_maps_tensor = torch.tensor(objects_grid_maps, dtype=torch.float)

        #multiply by 1000 so the input will be suitable for Llama model (receives only integers)
        objects_grid_maps_tensor_multiplied = torch.round(objects_grid_maps_tensor * 1000).to(torch.long)

        ###create a ground truth map
        # # **Calibrate Optical points with LIDAR**
        ###create the ground truth data
        # Rename categories to match grid map object names
        ground_truth['label'] = ground_truth['label'].replace({
            'vehicle.car': 'car',
            'vehicle.truck': 'truck',
            'human.pedestrian.adult': 'person'
        })
        
        ground_truth = ground_truth[ground_truth['label'].isin(['car', 'truck', 'person'])]
#         display(ground_truth)
        ###convert camera point to lidar point

        # Example usage:
        ground_truth_map, unique_labels = create_ground_truth_map(ground_truth)
        if ground_truth_map is None:
            print ("ground truth is empty, skipping to the next frame")
            next_scene = nusc.get('sample',sample_token)['next']
            continue
            
        ground_truth_tensor = torch.tensor(ground_truth_map).to(torch.float32)
        #pad the tensor
        padded_ground_truth_tensor = pad_tensor_to_target(ground_truth_tensor, (num_objects, x_size, y_size))
        padded_objects_grid_maps_tensor_multiplied = pad_tensor_to_target(objects_grid_maps_tensor_multiplied, (num_objects, x_size, y_size))
        padded_lidar_grid = pad_tensor_to_target(torch.tensor(lidar_grid).float().unsqueeze(0), (1, x_size, y_size))
        concat_lidar_opt = torch.cat([padded_lidar_grid, padded_objects_grid_maps_tensor_multiplied], dim=0)
        #add to the batch the data
        display(concat_lidar_opt.unsqueeze(0).size())
        display(training_batch.size())
        training_batch = torch.cat((training_batch,concat_lidar_opt.unsqueeze(0)), dim=0)
        label_batch = torch.cat((label_batch, padded_ground_truth_tensor.unsqueeze(0)), dim=0)
        
        #save the ego-pose and calibrated sensor data
        lidar_sample_data_token = my_sample['data']['LIDAR_TOP']
        lidar_sample_data = nusc.get('sample_data', lidar_sample_data_token)

        # 3. Get the ego_pose and calibrated_sensor records using tokens from the lidar_sample_data
        ego_pose_record = nusc.get('ego_pose', lidar_sample_data['ego_pose_token'])
        calibrated_sensor_record = nusc.get('calibrated_sensor', lidar_sample_data['calibrated_sensor_token'])
        sample_token_key = sample_token
        logs[training_batch.size()[0] -1] = [sample_token_key, ego_pose_record, calibrated_sensor_record]
        print ("next_scene is: ",next_scene)
        print ("---------")
        print (my_scene['last_sample_token'])
        
        # --- לוגיקת שמירת CHECKPOINT ---
        # נשמור את המצב הנוכחי כל SAVE_EVERY_N_SAMPLES דגימות
        # התנאי `training_batch.size(0) > 0` מוודא שלא נשמור לפני שעיבדנו משהו
        if training_batch.size(0) > 0 and training_batch.size(0) % SAVE_EVERY_N_SAMPLES == 0:
            print(f"\n💾 שומר checkpoint אחרי {training_batch.size(0)} דגימות מעובדות...")
            
            # המצב שנשמור הוא המצב של *האיטרציה הבאה*
            state_to_save = {
                'scene_idx': scene_idx,
                'next_scene': nusc.get('sample', sample_token)['next'],
                'training_batch': training_batch,
                'label_batch': label_batch,
                'logs': logs,
                'not_found_files': not_found_files,
                'scenes_with_missing_files': scenes_with_missing_files,
            }
            torch.save(state_to_save, CHECKPOINT_PATH)
            print(f"השמירה הושלמה. הקובץ נשמר ב: {CHECKPOINT_PATH}\n")
            print ("=================")
            print ("=================")
        display(next_scene)
        if next_scene == '' and my_scene['last_sample_token']:
            print ("finished one session, going to the next one")
            next_scene = None
            scene_idx +=1
            pbar.update(1)  # Update progress bar when moving to next scene
            print (fr'moving to another scene, added 1 to scene_idx now  is {scene_idx}')
            continue
        elif next_scene == my_scene['last_sample_token']:
            pbar.update(1)  # Update final progress
            scene_idx +=1
        else:
            next_scene = nusc.get('sample',sample_token)['next']
        print ("number of samples in training_batch:", training_batch.size(0))

✅ נמצא קובץ שמירה. טוען התקדמות מ: nuscenes_checkpoint1.pth
הריצה תמשיך מסצנה מספר 38.


Processing scenes:   3%|█▋                                                              | 1/39 [00:00<00:00, 51.13it/s]


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
def plot_optical_grid_map(grid_map):

    plt.figure(figsize=(8, 8))
    data = np.array(grid_map, dtype = np.float32)
    sns.heatmap(data, cmap='viridis', cbar_kws={'label': 'Values'})
    plt.title('Map Heatmap')
    plt.show()

In [11]:
import os

# קבל את תיקיית העבודה הנוכחית
current_directory = os.getcwd()

print(f"תיקיית העבודה הנוכחית היא: {current_directory}")

תיקיית העבודה הנוכחית היא: C:\Users\yuval\nuscenes-devkit


In [12]:
import json
output_filename = fr'log_data_{grid_size}_meters.json'

# פתח קובץ לכתיבה ושמור את המידע בפורמט JSON
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(logs, f, indent=4)

print(f"Data has been saved to '{output_filename}'")

Data has been saved to 'log_data_1_meters.json'


In [13]:
torch.save(training_batch, fr'C:\Users\yuval\training_batch_{grid_size}_meter_grid.pt')
torch.save(label_batch, fr'C:\Users\yuval\label_batch_{grid_size}_meter_grid.pt')

QA

In [14]:
sample_token = "ea0979999bf84ee6a6230b1bde7ec723"

In [15]:
keys = nusc.get('sample', sample_token)
keys

KeyError: 'ea0979999bf84ee6a6230b1bde7ec723'

In [ ]:
cam_front_key = nusc.get("sample", "ecdf7078afcf4949b7310611c60a1f80")['data']['CAM_FRONT']
nusc.get("sample_data", cam_front_key)

In [ ]:
nusc.get("sample_annotation", "bf497b43ec6e4937a9447e86035be504")

In [ ]:
pc = LidarPointCloud.from_file(fr"D:\Nuscenes\unified\samples\LIDAR_TOP\n015-2018-11-14-18-57-54+0800__LIDAR_TOP__1542193562548545.pcd.bin")
bin_pcd = pc.points.T
bin_pcd_only_axis = bin_pcd.reshape((-1, 4))[:, 0:3]

o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(bin_pcd_only_axis))
bin_pcd_with_intensity = bin_pcd.reshape((-1, 4))
intensity_values = bin_pcd_with_intensity[:,3]
bin_pcd_only_axis.shape
image_fullpath = fr"D:\Nuscenes\unified\samples\CAM_BACK_LEFT\n015-2018-11-14-18-57-54+0800__CAM_BACK_LEFT__1542193562547423.jpg"
image = cv2.imread(image_fullpath)
    
  # Display the image
  # cv2_imshow(image)
################################
# Load the model for detecting potential annotations


# Convert the NumPy array to a PIL Image object
image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) #added color conversion
results = detector(image_pil)
results

In [ ]:
label_batch[2,1,:,:]

# **Save the dataset**

In [ ]:
data = torch.load('label_batch_only_required_scenes_completion.pt')

In [ ]:
data.size()

In [ ]:
import os
# Print working directory
print("Current working directory:", os.getcwd())

# Search for the file in current directory and subdirectories
import glob
print("\nSearching for file...")
matching_files = glob.glob('**/training_batch_only_required_scenes.pt', recursive=True)
print(f"Found matches: {matching_files}")

# Alternative search using system command
print("\nAlternative search:")
!find / -name "training_batch_trainval10.pt" -type f 2>/dev/null | head -n 10

In [ ]:
print (len(scenes_with_missing_files), scenes_with_missing_files)
print (not_found_files)